In [23]:
import random
import tensorflow as tf
import numpy as np
import pandas as pd

import nltk
from gensim.models import Word2Vec
from gensim.utils import tokenize

from collections import defaultdict

In [74]:
# Hyper Parameter

EPOCHS = 50
NUM_WORDS = 30000

In [51]:
origin_path = 'c:/dev/dataset/wikihowAll_pre.csv'

origin_data = pd.read_csv(origin_path)

In [52]:
origin_data = origin_data.drop(origin_data.columns[0], axis=1)
origin_data

,headline,title,text
0,Keep related supplies in the same area.Make an...,How to Be an Organized Artist1,If you're a photographer keep all the necessa...
1,Create a sketch in the NeoPopRealist manner of...,How to Create a Neopoprealist Art Work,See the image for how this drawing develops s...
2,Get a bachelor’s degree.Enroll in a studio-bas...,How to Be a Visual Effects Artist1,It is possible to become a VFX artist without...
3,Start with some experience or interest in art....,How to Become an Art Investor,The best art investors do their research on t...
4,Keep your reference materials sketches article...,How to Be an Organized Artist2,As you start planning for a project or work y...
...,...,...,...
214289,Consider changing the spelling of your name.Av...,How to Pick a Stage Name3,If you have a name that you like you might fi...
214290,Try out your name.Don’t legally change your na...,How to Pick a Stage Name4,Your name might sound great to you when you s...
214291,Understand the process of relief printing.Exam...,How to Identify Prints1,Relief printing is the oldest and most tradit...
214292,Understand the process of intaglio printing.Lo...,How to Identify Prints2,"Intaglio is Italian for ""incis­ing"" and corre..."


In [55]:
origin_data = origin_data.dropna()

In [56]:
x, y = origin_data['text'], origin_data.headline

In [22]:
# tokenized dictionary
def tokenized_to_dic(df) :
    x_token = defaultdict(int)
    y_token = defaultdict(int)

    for idx in range(len(df))



([], [])

In [39]:
# encoder & decoder
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        # 입력이 one-hot-encdoing 형식으로 들어오면 embedding을 수행을 먼저
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        # 그리고 LSTM에서 hidden state을 출력으로 던져주어야 이것을 활용해서 return을 활용할 수 있다...
        # return_sequence=True : 디코더에서 나오는 것 하나하나 알아야 하기에...
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        # return_sequences = True -> attention 하기 위해서

    # 인코더의 경우에는 입력이 들어오면 이것을 바탕으로 히든, 셀 스테이트를 하도록 ...
    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        # h,c는 test를 할 경우에는 다음으로 넘겨 주어야 forward  로 계산을 하면서 나갈 수 있으니..
        H, h, c = self.lstm(x)
        return H, h, c
    
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        
        #****Attention을 추가함!!!! 
        self.att = tf.keras.layers.Attention()
        
        # 최종 단어 단에서 어떤 것을 할 것인지..선택..
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        # ***처음 들어오는 인코더 단에서 넘어오는 것들을 받고,,
        x, s0, c0, H = inputs
        x = self.emb(x)
        # ****S는 hidden state를 모두 모아둔 부분..--> 쿼리로 사용을 할 것임....--->하나 앞선 시간을 사용을 해서..
        S, h, c = self.lstm(x, initial_state=[s0, c0])
        # ***쿼리로 사용을 할 것임....--->하나 앞선 시간을 사용을 해서..(1차원을 3차원으로 확장) & 마지막 히든은 제외...해서 :-1까지..
        S_ = tf.concat([s0[:, tf.newaxis, :], S[:, :-1, :]], axis=1)
        # ****키와 value를 계산하고,,,
        A = self.att([S_, H])
        y = tf.concat([S, A], axis=-1)
        
        return self.dense(y), h, c

In [40]:
# seq2seq
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        # 1) 가장 기본적으로 필요한 Encoder, Deoder, sos, eos에 관련된 부분을 지정을 함!!!!
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos
    
    # 연결에서 구성 --> 학습을 위해서는 입력/출력 모두 알아야 하기에 x,y = input으로 받음
    # 그리고 디코더에 입력을 넣어주어야 하기에..
    def call(self, inputs, training=False, mask=None):
       # (학습 과정)
        if training is True:
            # 학습을 위해서는 입력/출력 모두 알아야 하기에 x,y = input으로 받음
            # 그리고 디코더에 입력을 넣어주어야 하기에..
            x, y = inputs
            # encoder에 입력  x를 넣어서 나온 hidden state, cell  (LSTM으로 구현이 되어서..)
            H, h, c = self.enc(x)
            # 최종 출력 y를 하도록...
            y, _, _ = self.dec((y, h, c, H))
            return y
        # (Test 과정) --> 그러니 정답  y는 없음..
        else:
            x = inputs
            H, h, c = self.enc(x)
            # 뒤에 디커더 부분에 입력을 넣어 주는 부분이 달라지게 된다!!!! --> sos  를 넣어준다.
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))

            # 최대 seq는 64길이까지만..
            seq = tf.TensorArray(tf.int32, 64)
            
            # tf의  for loop으로 사용을 해서 최대 64까지 
            for idx in tf.range(64):
                # for 에서 처음에는 처음으로 만들어준 sos를 decoder에 입력으로 넣어주고 --> self.dec(y,h,c,H)을 해서
                # 출력인 y와 hidden state, cell state를 출력으로 준다.
                y, h, c = self.dec([y, h, c, H])
                # 가장 큰 y값에 대한 index를 얻어오게 되는 과정
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                # reshape를 하면서 batch를 사용을 하기 위해서...
                y = tf.reshape(y, (1, 1))
                # seq를 하나씩 받으면서 처리하기 위해서...
                seq = seq.write(idx, y)
                # eos일 때 까지 for lopp순환
                if y == self.eos:
                    break
            # 
            return tf.reshape(seq.stack(), (1, 64))

In [41]:
# implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    # 마지맞 eos는 있고, 처음 sos 는 없는 것...
    output_labels = labels[:, 1:]
    # 처음 sos는 포함이 되고, 마지막 eos는 빼고...
    shifted_labels = labels[:, :-1]
    # 그래서 위의 시프트 된 것들이 아래의 model의 학습 과정에 들어가게 된다!!!!!
    with tf.GradientTape() as tape:
        # 위의 output_labels, shifted_labels 이 들어가게 된다...
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)


In [70]:
x_sub = list(x[:5000].apply(lambda x : x+'\n'))
y_sub = list(y[:5000].apply(lambda x : '\t'+x+'\n'))

In [71]:
num_sample = len(x_sub)

# 질문을 보고 한 번 데이터 셋을 섞어준다..
perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)
# 섞으면서 train/ test로 구분을 하기 위한 것..
train_q = list()
train_a = list()
test_q = list()
test_a = list()

# 질문과 답변에 대한 것을 돌면서...
for idx, qna in enumerate(zip(x_sub, y_sub)):
    q, a = qna
    # 5/1은 test, 5/4는 train
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

# 문장에서 짤라주는 것에 대한 세팅...
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

# 입력은 뒤로...패딩..
x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
# 출력은 앞에로 패딩..
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

In [72]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [73]:
# train loop
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 4.351285934448242, Accuracy: 42.09489822387695
Epoch 2, Loss: 3.7813518047332764, Accuracy: 44.473228454589844
Epoch 3, Loss: 3.649198532104492, Accuracy: 45.32344055175781
Epoch 4, Loss: 3.485776424407959, Accuracy: 46.4370002746582
Epoch 5, Loss: 3.342334270477295, Accuracy: 47.613075256347656
Epoch 6, Loss: 3.2132675647735596, Accuracy: 48.48126220703125
Epoch 7, Loss: 3.0872557163238525, Accuracy: 49.288883209228516
Epoch 8, Loss: 2.976085662841797, Accuracy: 50.017189025878906
Epoch 9, Loss: 2.8648040294647217, Accuracy: 50.814266204833984
Epoch 10, Loss: 2.75075101852417, Accuracy: 51.64415740966797
Epoch 11, Loss: 2.6351394653320312, Accuracy: 52.297447204589844
Epoch 12, Loss: 2.524951219558716, Accuracy: 53.15352249145508
Epoch 13, Loss: 2.4113383293151855, Accuracy: 54.142051696777344
Epoch 14, Loss: 2.3014323711395264, Accuracy: 55.19661331176758
Epoch 15, Loss: 2.189403772354126, Accuracy: 56.438720703125
Epoch 16, Loss: 2.0746588706970215, Accuracy: 57.90588

KeyboardInterrupt: 

In [ ]:
# test
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('Question : ', test_text)
    print('Prediction Answer: ', texts)
    print('Real Answer : ', gt_text)